In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# carga de gobienos locales 
df_goblocales = gpd.read_file("../GobiernosLocales/GobiernosLocales.shp")
df_goblocales.rename(columns={'Provincia':'provincia', 'Departamen':'departamen', 'Municipio':'municipio', 'UTA2020':'uta2020'}, inplace=True)

In [3]:
# carga de los datos de la dimensión Sistema Demográfico
df_pob_data = pd.read_csv("../BD_Originales/SistemaDemografico_GobiernosLocales.csv")
df_pob_data.rename(columns={'coduta':'uta2020'}, inplace=True)

df_pob_data.uta2020 = df_pob_data.uta2020.astype(str)
for i in range(len(df_pob_data)):
    if len(df_pob_data.loc[i, 'uta2020']) == 8:
        df_pob_data.loc[i, 'uta2020'] = '0'+df_pob_data.loc[i, 'uta2020']

In [4]:
# carga capa de departamentos
df_deptos = gpd.read_file("../CapasGeograficasOriginales/departamento.geojson")
df_deptos.rename(columns={'in1':'id_cod_dep'}, inplace=True)

In [5]:
# carga capa de radios censales
df_pob_d = gpd.read_file("../CapasGeograficasOriginales/radiocensales2010.geojson")

## Superficie

In [6]:
# cálculo del valor de superficie de gobiernos locales
df_goblocales = df_goblocales.to_crs("EPSG:3035")
df_goblocales['area_k2'] = df_goblocales.area / 1000000
df_goblocales = df_goblocales.to_crs("EPSG:4326")

In [7]:
# cálculo de área por depto
df_deptos = df_deptos.to_crs("EPSG:3035")
df_deptos['area_k2_depto'] = df_deptos.area / 1000000
df_deptos = df_deptos.to_crs("EPSG:4326")
# cálculo de área por pcia
df_deptos['cpr'] = df_deptos.id_cod_dep.str[:2]
df_sup_pcia = df_deptos.groupby('cpr')[['area_k2_depto']].sum().reset_index().rename(columns={'area_k2_depto':'area_k2_pcia'})

# Cálculo de población 

In [8]:
# creación de una variable uniendo los datos poblacionales a los gobiernos locales
df_goblocales = df_goblocales.merge(df_pob_data[['uta2020', 'dem_pobt']], on='uta2020', how='left')
df_goblocales.dem_pobt.fillna(0, inplace=True)

In [9]:
# id de deptos
df_pob_d['id_cod_dep'] = df_pob_d['prov'] +df_pob_d['depto']
# calculo de población por depto
df_pob_depto = df_pob_d.groupby('id_cod_dep')[['totalpobl']].sum().reset_index().rename(columns={'totalpobl':'totalpobl_depto'})
# calculo de población por pcia
df_pob_pcia = df_pob_d.groupby('prov')[['totalpobl']].sum().reset_index().rename(columns={'prov':'cpr', 'totalpobl':'totalpobl_pcia'})

## Porcentaje superficie y población sobre el depto

In [10]:
# id de deptos
df_goblocales['id_cod_dep'] = df_goblocales['cpr'] + df_goblocales['cde']

In [11]:
# cruce de gob locales y datos de superficie por depto
df_goblocales_deptos = df_goblocales.merge(df_deptos[['id_cod_dep', 'area_k2_depto']], on='id_cod_dep', how="left")
df_goblocales_deptos.rename(columns={'area_k2_depto':'area_depto'}, inplace=True)

In [12]:
# cruce de gob locales y datos de población por depto
df_goblocales_deptos = df_goblocales_deptos.merge(df_pob_depto, on='id_cod_dep', how="left")
df_goblocales_deptos.rename(columns={'totalpobl_depto':'dem_pobt_depto'}, inplace=True)

In [13]:
# cálculo del porcentaje de población y superficie del gobierno local sobre el departamento
df_goblocales_deptos['areap_dept'] = df_goblocales_deptos['area_k2'] / df_goblocales_deptos['area_depto']
df_goblocales_deptos['pobp_dept'] = df_goblocales_deptos['dem_pobt'] / df_goblocales_deptos['dem_pobt_depto']
df_goblocales_deptos.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,Longitud,buffer,geometry,area_k2,dem_pobt,id_cod_dep,area_depto,dem_pobt_depto,areap_dept,pobp_dept
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,-58.37153965302685,0,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5...",17.887891,205886.0,02007,17.911135,220591.0,0.998702,0.933338
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,-58.39489181181946,0,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5...",6.335529,157932.0,02014,6.322778,225970.0,1.002017,0.698907
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,-58.40265145122171,0,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5...",6.385607,187537.0,02021,6.385607,NaN,1.000000,NaN
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,-58.38745506898847,0,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6...",22.763971,218245.0,02028,22.710860,NaN,1.002339,NaN
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,-58.42057219028574,0,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5...",6.660127,179005.0,02035,6.660127,NaN,1.000000,NaN


## Porcentaje superficie y población sobre provincia

In [14]:
# cruce de gob locales y datos de superficie por pcia
df_goblocales_pcias = df_goblocales_deptos.merge(df_sup_pcia, on='cpr', how="left", )
df_goblocales_pcias.rename(columns={'area_k2_pcia':'area_pcia'}, inplace=True)

In [15]:
# cruce de gob locales y datos de población por pcia
df_goblocales_pcias = df_goblocales_pcias.merge(df_pob_pcia, on='cpr', how="left")
df_goblocales_pcias.rename(columns={'totalpobl_pcia':'dem_pobt_pcia'}, inplace=True)

In [16]:
# cálculo del porcentaje de población y superficie del gobierno local sobre la provincia
df_goblocales_pcias['areap_pcia'] = df_goblocales_pcias['area_k2'] /df_goblocales_pcias['area_pcia']
df_goblocales_pcias['pobp_pcia'] = df_goblocales_pcias['dem_pobt'] /df_goblocales_pcias['dem_pobt_pcia']
df_goblocales_pcias.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,dem_pobt,id_cod_dep,area_depto,dem_pobt_depto,areap_dept,pobp_dept,area_pcia,dem_pobt_pcia,areap_pcia,pobp_pcia
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,205886.0,02007,17.911135,220591.0,0.998702,0.933338,205.943031,2890151.0,0.086858,0.071237
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,157932.0,02014,6.322778,225970.0,1.002017,0.698907,205.943031,2890151.0,0.030764,0.054645
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,187537.0,02021,6.385607,NaN,1.000000,NaN,205.943031,2890151.0,0.031007,0.064888
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,218245.0,02028,22.710860,NaN,1.002339,NaN,205.943031,2890151.0,0.110535,0.075513
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,179005.0,02035,6.660127,NaN,1.000000,NaN,205.943031,2890151.0,0.032340,0.061936


## Limpieza

In [17]:
# eliminación de columnas
df_goblocales_pcias = df_goblocales_pcias.drop(columns=['dem_pobt', 'id_cod_dep', 'area_depto', 'dem_pobt_depto', 'area_pcia', 'dem_pobt_pcia'])

# cambio de nombre de los campos calculados
df_goblocales_pcias.rename(columns={'area_k2':'k2_sup_tot', 
        'areap_dept':'pc_sup_xde', 'pobp_dept':'pc_hab_xde', 
        'areap_pcia':'pc_sup_xpr', 'pobp_pcia':'pc_hab_xpr',}, inplace=True)

# formateo de los campos numéricos
df_goblocales_pcias.k2_sup_tot = round(df_goblocales_pcias.k2_sup_tot, 2)
df_goblocales_pcias.pc_sup_xde = df_goblocales_pcias.pc_sup_xde * 100
df_goblocales_pcias.pc_sup_xde = round(df_goblocales_pcias.pc_sup_xde, 2)
df_goblocales_pcias.pc_hab_xde = df_goblocales_pcias.pc_hab_xde * 100
df_goblocales_pcias.pc_hab_xde = round(df_goblocales_pcias.pc_hab_xde, 2)
df_goblocales_pcias.pc_sup_xpr = df_goblocales_pcias.pc_sup_xpr * 100
df_goblocales_pcias.pc_sup_xpr = round(df_goblocales_pcias.pc_sup_xpr, 2)
df_goblocales_pcias.pc_hab_xpr = df_goblocales_pcias.pc_hab_xpr * 100
df_goblocales_pcias.pc_hab_xpr = round(df_goblocales_pcias.pc_hab_xpr, 2)

# relleno de valores nulos de campos numéricos
df_goblocales_pcias.k2_sup_tot.fillna(0, inplace=True)
df_goblocales_pcias.pc_sup_xde.fillna(0, inplace=True)
df_goblocales_pcias.pc_hab_xde.fillna(0, inplace=True)
df_goblocales_pcias.pc_sup_xpr.fillna(0, inplace=True)
df_goblocales_pcias.pc_hab_xpr.fillna(0, inplace=True)

# asignación de los valores a "null" de los gobiernos locales sin polígono (buffer == 1)
df_goblocales_pcias.loc[df_goblocales_pcias['buffer'] == 1, ['k2_sup_tot', 'pc_sup_xde', 'pc_hab_xde', 'pc_sup_xpr', 'pc_hab_xpr']] = float("NaN")

## Exportar

In [18]:
# exportación de los datos geenrados a csv
columns_list = ['uta2020', 'k2_sup_tot', 'pc_sup_xde', 'pc_hab_xde', 'pc_sup_xpr', 'pc_hab_xpr']
df_goblocales_pcias[columns_list].to_csv("../BD_Procesada/InsercionTerritorial_GobiernosLocales.csv", encoding='utf-8')

In [19]:
# exportación de la capa de gobiernos locales con los datos generados
schema_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 'tipgobloc':'str',
                    'uta2020':'str','k2_sup_tot':'float:8.2', 'pc_sup_xde':'float:8.2', 'pc_hab_xde':'float:8.2', 
                    'pc_sup_xpr':'float:8.2', 'pc_hab_xpr':'float:8.2'}}
df_goblocales_pcias.crs = 'epsg:4326'
df_goblocales_pcias[['provincia', 'departamen', 'municipio', 'tipgobloc']+ columns_list +['geometry']].to_file("../CapasGeograficasProcesadas/InsercionTerritorial_GobiernosLocales.shp", 
           driver='ESRI Shapefile', 
           schema=schema_dic,
           encoding='utf-8')